In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
df.info()

* Observations:
* 1460 rows.
* 81 Variables.
* Three(3) vars float64.
* 35 vars int64.
* 43 vars objects.

In [ ]:
#df.select_dtypes("number").columns --> number get data type int and float

In [ ]:
df.select_dtypes("int").columns

In [ ]:
df.select_dtypes("float").columns

In [ ]:
df.select_dtypes("object").columns

* selected the features that I have considered important features that to not have a wide table as an output of describe function.

In [ ]:
cols_sum_stat = [
    "LotFrontage",
    "LotArea",
    "MasVnrArea",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "GrLivArea",
    "BsmtFullBath",
    "FullBath",
    "BedroomAbvGr",
    "KitchenAbvGr",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "SalePrice",
]

df[cols_sum_stat].describe().round()


* Many features have zero ,SalePrice ranges from 34900 to 755000, mean of 180921 and median or 50% of 163000 ,we have to check the GrLivArea ranges from 334 to 5642.
* LotArea ranges from 1300 to 215245.
* BedroomAbvGr ranges from zero to eight.

###  check  missing values

In [ ]:
#missing data
total = df.isna().sum()[df.isna().sum()>0].sort_values(ascending=False)
percent = (df.isna().sum()[df.isna().sum()>0]/df.isna().sum()[df.isna().sum()>0].count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

* the features in "cols_corr_out" list, i think it will be more readable!

In [ ]:
cols_corr_out = [
    "Id",
    "MSSubClass",
    "OverallCond",
    "Fireplaces",
    "PoolArea",
    "MiscVal",
    "MoSold",
    "YrSold",
    "BsmtHalfBath",
    "BsmtFinSF2",
    "LowQualFinSF",
    "EnclosedPorch",
]

In [ ]:
corr_mat = (
    df.loc[:, ~df.columns.isin(cols_corr_out)]
    .select_dtypes("int64")
    .corr()
)
corr_mat[["SalePrice"]].sort_values("SalePrice", ascending=False).style.format(
    precision=3
).background_gradient(cmap="Greys")

* Correlation between target variable and other variables:
* OverallQual: 0.790
* GrLivArea: 0.708
* GarageCars: 0.640
* GarageArea: 0.623
* TotalBsmtSF: 0.613
* 1stFlrSF: 0.605
* FullBath: 0.560
* TotRmsAbvGrd:0.533
* YearBuilt: 0.522
* YearRemodAdd:0.507


In [ ]:
mask = np.triu(np.ones_like(corr_mat.corr()))
fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(
    corr_mat,
    square=False,
    cmap="Greys",
    mask=mask,
    annot=True,
    annot_kws={"size": 8},
    fmt=".2f",
)
plt.xticks(fontsize=7)
plt.yticks(fontsize=7)
plt.show()

* many variables are correlated together, and most of them are practically obvious to be correlated such like:
* "1stFlSf" and "TotalBsmtSF": 0.82

* "TotalRmsAbvGrd" and "GrLivArea": 0.83

* "GarageArea" and "GarageCars": 0.88

* "GrLivArea" and "2ndFlrSF": 0.69

* "TotalRmsAbvGrd" and "BedroomAbvGr": 0.68

* "BsmtFullBath" and "BsmtFinSF1": 0.65

* "FullBath" and "GrLivArea": 0.63

* "YearBuilt" and "OverallQual": 0.57

* "YearRemodAdd" and "YearBuilt": 0.59

* "FullBath" and "YearBuilt": 0.59


###  Feature-Engineering


* Total Square Feet

* I will add up the living areas that above and below ground level to get the total living area

In [ ]:
df["Total_Square_Feet"] = (
    df["GrLivArea"] + df["TotalBsmtSF"]
)
df["Total_Square_Feet"]

#### Total number of Bathrooms

* Total number of Bathrooms
* There are four(4) Features of bathrooms, Full bathrooms, and Half Bathrooms, however, I will consider them as one type of "Bathroom" and add them together!

In [ ]:
df["Total_number_of_Bathrooms"] = (
    df["FullBath"]
    + (df["HalfBath"])
    + df["BsmtFullBath"]
    + (df["BsmtHalfBath"])
)

df["Total_number_of_Bathrooms"].describe().round()

* min number of bathrooms: 1

* max number of bathrooms: 6

#### House Age

* i will be using YearBuilt and YearSold to find out the House Age, however I will create another house Age variable using YearRemodAdd.

In [1]:
df["House_Age"] = df["YrSold"] - df["YearBuilt"]

df["House_Age_YearRemodAdd"] = (
    df["YrSold"] - df["YearRemodAdd"]
)

df[["House_Age"]].describe().round(), df[
    ["House_Age_YearRemodAdd"]
].describe().round()

NameError: name 'df' is not defined

* mean house age: 23.0

* median house age: 35.0

* min house age: Zero(0)

* max house age: 136.0

* Let us check the min of -One(-1) in House_Age_YearRemodAdd.
maybe this house has been sold while the house is under construction or renovation SO let  check

In [ ]:
df[df["House_Age"] == 0].loc[
    :, ["YearBuilt", "YearRemodAdd", "YrSold", "House_Age", "House_Age_YearRemodAdd"]
].sort_values("House_Age_YearRemodAdd").head()

* We do not know whether this house with -One(-1) has been sold while it was under renovation, or is a data entry issue

###  Neighborhood

* For plotting purpose, I am going to select five(5) out of the observations in "Neighborhood" feature, and lump the other observations under one category named as "other".

In [ ]:
(df[["Neighborhood"]]
     .value_counts()
     .nlargest(20)
     .reset_index()
     .loc[:, "Neighborhood"].to_list())


In [ ]:
top_5_Neighborhood = [
    "Edwards",
    "NridgHt",
    "NoRidge",
    "OldTown",
    "CollgCr"
]

In [ ]:
df["Neighborhood"] = np.where(
    df["Neighborhood"].isin(top_5_Neighborhood),
    df["Neighborhood"],
    "Other",
)

##### Analyzing the Target variable "SalePrice"

In [ ]:
df[["SalePrice"]].describe().round()

In [ ]:
px.histogram(df,x='SalePrice')

* Data is positively skewed. The mean is greater than the median 
* he outliers located in the right side, We may use log transformation to reduces the skewness 
* The majority of data points are clustered between 100K and 200K 

In [ ]:
df["SalePriceLog"] = np.log(df["SalePrice"])
df["GrLivAreaLog"] = np.log(df["GrLivArea"])
df["LotAreaLog"] = np.log(df["LotArea"])
df[
    ["SalePrice", "SalePriceLog", "GrLivArea", "GrLivAreaLog", "LotArea", "LotAreaLog"]
]

###### SalePrice" vs "Total_Square_Feet" by  SaleCondition

In [ ]:
#to_plot(
 #   df=df,
  #  x_var="Total_Square_Feet",
   # y_var="SalePrice",
    #hue_var="SaleCondition",
    #palette_set=my_palette,
#)
px.line(df,x="Total_Square_Feet",y="SalePrice",color='SaleCondition')

* The majority of sales condition is "Normal" sales condition

* We have extreme two(2) outliers related to Partial sales condition that we need to check them

* Mainly house area Square Feet is one of the important features to determine the sale price

* We know from the correlation matrix that GrLivAra and SalePrice vars are highly correlated 0.709


###### SalePrice"-vs-"House_Age"

In [ ]:
px.line(df,x="House_Age",y="SalePrice")

* We can notice kind of clusters
- From ~0 to ~20.
- From ~25 to ~60.
- From ~80 to ~100.

* We have many data points above ~100 years with a high price, that is assumingly connected to another correlated feature like:
- OverallQual-SalePrice: 0.791
- GrLivAra-SalePrice: 0.709
- YearBuilt-SalePrice: 0.523
- YearRemodAdd-SalePrice: 0.507

###### "SalePrice" vs "Total_Square_Feet" by "Neighborhood"

In [ ]:
px.line(df,x="Total_Square_Feet",y="SalePrice",color='Neighborhood')

*  It is clear that the Neighborhood feature has an important role to play in determining the sale price

* The two(2) extreme data points are related to Edwards Neighborhood category

* The data points representing the highest price are related to NoRidge Neighborhood category



###### create a table to have more information

In [ ]:
df[[
        "SalePrice",
        "GrLivArea",
        "LotArea",
        "PoolArea",
        "Total_Square_Feet",
        "YearBuilt",
        "FullBath",
        "KitchenQual",
        "OverallQual",
        "ExterCond",
        "MSSubClass",
        "Neighborhood",
        "Utilities",
        "Foundation",
    ]
].nlargest(columns="Total_Square_Feet", n=10).style.background_gradient()

* Through the above table we see that mainly the NoRidge Neighborhood category has defeats the Edwards category, although the PoolArea and FullBath have a little influence over that matter as well.

# Relationship with categorical variables

* "SalePrice" vs "SaleCondition" by "RoofStyle"

In [ ]:
px.line(df,x="SaleCondition",y="SalePrice",color='RoofStyle')

* SalePrice" vs "OverallQual" by "SaleCondition"

In [ ]:
df[["OverallQual"]].value_counts(normalize=True).round(
    3
).reset_index().rename(columns={0: "Percentage"}).style.background_gradient()

In [ ]:
px.scatter(df,x="OverallQual",y="SalePrice")

* About 70% of the data points in the OverallQual var are five(5), Six(6) and Seven(7) collectively


###### "SalePrice" vs "KitchenQual" by "Electrical"

In [ ]:
px.bar(df,x="KitchenQual",y="SalePrice",color='Electrical')

* SBrkr: Standard Circuit Breakers & Romex.
* FuseA: Fuse Box over 60 AMP and all Romex wiring (Average).
* FuseF: 60 AMP Fuse Box and mostly Romex wiring (Fair).
* FuseP: 60 AMP Fuse Box and mostly knob & tube wiring (poor.
* Mix: Mixed.


* Standard Circuit Breaker based power protection system is more desirable by the buyer, and if we go deeply into the analysis we would find that all modern houses have equipped with this kind of power protection system!

#### SalePrice" relationship with various variable using FacetGrid Scatter Plot

In [ ]:
def grid_plot(df, col_var, x_var, y_var, paint):

    f_grid = sns.FacetGrid(
        df,
        col=col_var,
        col_wrap=3,
        height=3,
        aspect=1.3,
        sharex=True,
        sharey=True,
    )

    f_grid.map_dataframe(
        sns.scatterplot,
        x=x_var,
        y=y_var,
        color=paint,
        alpha=0.5,
    ).set(ylim=(-0, None), xlim=(-0, None))

    f_grid.set_titles(col_template="{col_var} ({col_name})", size=10, c="#386087")
    f_grid.set_yticklabels(fontsize=8)
    f_grid.set_xticklabels(fontsize=8)
    for ax in f_grid.axes.flat:
        ax.yaxis.set_major_formatter(ticker.EngFormatter())
        ax.xaxis.set_major_formatter(ticker.EngFormatter())

    f_grid.set_xlabels(label=x_var + " (square feet)", size=10)
    f_grid.set_ylabels(label="$ " + y_var, size=10)
    return plt.show()



#### "SalePrice" vs "Total_Square_Feet" by "Foundation"

In [ ]:
grid_plot(
    df,
    col_var="Foundation",
    x_var="Total_Square_Feet",
    y_var="SalePrice",
    paint="#008080",
)

* "Gas forced warm air furnace" almost dominate all types of heating mentioned in this feature.

* 18 data points for "Gas hot water or steam heat" and one(1) data point for "Floor Furnace"


In [ ]:
pair_cols = [
    "SalePrice",
    "Total_Square_Feet",
    "OverallQual",
    "TotalBsmtSF",
    "FullBath",
    "MasVnrArea",
]

##### Plot pairwise relationships of the selected Six variables in the dataset

In [ ]:
sns.pairplot(
    usa_housing_df[pair_cols],
    hue="OverallQual",
    palette=my_palette,
    kind="scatter",
    diag_kind="kde",
    height=1.5,
    aspect=1.5,
    plot_kws={"alpha": 0.7},
)
plt.show()

* * If we can deal with the outliers in a proper way, And consider the imbalanced proportions of observations in may features, we would have a big chance to build a solid model, however, we know that the house price is not related to the feature of this dataset, and the economy system and government decisions and legislation play a crucial role in this mater.